In [33]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from torchsummary import summary

In [2]:
data = pd.read_csv("diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
X = data.drop(columns=['Outcome', 'BMI']).values
y = data['BMI'].values.astype(np.float32)

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [6]:
full_data = np.hstack((X, y.reshape(-1, 1)))
X_seq, y_seq = split_sequences(full_data, n_steps=5)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [8]:
X_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)
X_test = torch.tensor(X_test, dtype=torch.float32).permute(0, 2, 1)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [9]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [30]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=7, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 5, 64)
        self.fc2 = nn.Linear(64, 1)  # 회귀 출력

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [31]:
model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []

# Training loop
for epoch in range(30):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Loss: {train_losses[-1]:.4f}")

Epoch 1, Loss: 1023.1285
Epoch 2, Loss: 652.0526
Epoch 3, Loss: 146.6936
Epoch 4, Loss: 85.0375
Epoch 5, Loss: 88.8167
Epoch 6, Loss: 74.7179
Epoch 7, Loss: 70.6883
Epoch 8, Loss: 69.1646
Epoch 9, Loss: 65.1167
Epoch 10, Loss: 61.7212
Epoch 11, Loss: 60.8245
Epoch 12, Loss: 57.0802
Epoch 13, Loss: 56.0307
Epoch 14, Loss: 54.9557
Epoch 15, Loss: 54.2121
Epoch 16, Loss: 53.1874
Epoch 17, Loss: 53.5585
Epoch 18, Loss: 50.4808
Epoch 19, Loss: 47.5025
Epoch 20, Loss: 47.9799
Epoch 21, Loss: 47.8248
Epoch 22, Loss: 56.3585
Epoch 23, Loss: 47.5935
Epoch 24, Loss: 44.7060
Epoch 25, Loss: 44.5140
Epoch 26, Loss: 52.1641
Epoch 27, Loss: 44.0444
Epoch 28, Loss: 41.6512
Epoch 29, Loss: 40.9895
Epoch 30, Loss: 41.0357


In [34]:
model.eval()
preds = []
actuals = []
with torch.no_grad():
    for xb, yb in test_loader:
        out = model(xb)
        preds.extend(out.squeeze().cpu().numpy())
        actuals.extend(yb.squeeze().cpu().numpy())

mse = mean_squared_error(actuals, preds)
r2 = r2_score(actuals, preds)
print(f"Test MSE: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Test MSE: 57.6926
R² Score: 0.1544
